<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/143_B2B_Sales_Agent_Claude_Langchain_01_Analysis_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
LangChain Analysis Agent - Identifies pain points and opportunities using LLM chains

This agent demonstrates:
- LangChain LLM integration
- Prompt templates and chains
- Structured output with Pydantic models
- Error handling and validation
"""

import logging
from typing import List, Dict, Any
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel as PydanticV1BaseModel
from langchain_models import (
    CompanyInfo, AnalysisResult, PainPoint, Opportunity,
    PainPointSeverity, OpportunityPriority
)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class AnalysisInput(PydanticV1BaseModel):
    """Input schema for analysis"""
    company_info: str

class LangChainAnalysisAgent:
    """
    LangChain Analysis Agent that uses LLM chains to analyze company information

    This demonstrates:
    - LangChain LLM integration
    - Prompt templates for structured analysis
    - Chain composition for complex workflows
    - Structured output with Pydantic models
    """

    def __init__(self, agent_id: str = "langchain_analysis_agent", use_mock: bool = True):
        self.agent_id = agent_id
        self.logger = logging.getLogger(f"{__name__}.{agent_id}")
        self.use_mock = use_mock

        if use_mock:
            # Use mock LLM for demonstration (no API key needed)
            self.llm = None
            self.logger.info("Using mock LLM for demonstration")
        else:
            # Initialize OpenAI LLM (requires API key)
            self.llm = ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.1,
                max_tokens=1000
            )
            self.logger.info("Using OpenAI LLM")

        # Initialize prompt templates
        self._setup_prompt_templates()

        # Initialize chains
        self._setup_chains()

        self.logger.info(f"LangChain Analysis Agent initialized: {agent_id}")

    def _setup_prompt_templates(self):
        """Set up prompt templates for analysis"""

        # Pain point analysis template
        self.pain_point_template = PromptTemplate(
            input_variables=["company_info"],
            template="""
Analyze the following company information and identify potential pain points:

Company Information:
{company_info}

Please identify pain points in the following format:
- Category: [category name]
- Description: [description]
- Severity: [low/medium/high/critical]
- Evidence: [supporting evidence]
- Solution: [potential solution]

Focus on business challenges, operational issues, and growth constraints.
Return up to 3 most relevant pain points.
"""
        )

        # Opportunity analysis template
        self.opportunity_template = PromptTemplate(
            input_variables=["company_info"],
            template="""
Analyze the following company information and identify potential opportunities:

Company Information:
{company_info}

Please identify opportunities in the following format:
- Category: [category name]
- Description: [description]
- Priority: [low/medium/high/urgent]
- Evidence: [supporting evidence]
- Value: [potential value]

Focus on growth opportunities, efficiency improvements, and market expansion.
Return up to 3 most relevant opportunities.
"""
        )

        # Strategy recommendation template
        self.strategy_template = PromptTemplate(
            input_variables=["pain_points", "opportunities"],
            template="""
Based on the following pain points and opportunities, recommend a sales approach:

Pain Points:
{pain_points}

Opportunities:
{opportunities}

Recommend one of these approaches:
1. pain_point_focused - Address critical challenges first
2. opportunity_focused - Leverage growth potential
3. relationship_building - General relationship building

Provide your recommendation and reasoning.
"""
        )

    def _setup_chains(self):
        """Set up LangChain chains"""
        if not self.use_mock:
            self.pain_point_chain = LLMChain(
                llm=self.llm,
                prompt=self.pain_point_template,
                output_key="pain_points"
            )

            self.opportunity_chain = LLMChain(
                llm=self.llm,
                prompt=self.opportunity_template,
                output_key="opportunities"
            )

            self.strategy_chain = LLMChain(
                llm=self.llm,
                prompt=self.strategy_template,
                output_key="strategy"
            )

    def analyze_company(self, company_info: CompanyInfo) -> AnalysisResult:
        """
        Analyze company information using LangChain LLM chains

        Args:
            company_info: CompanyInfo object from Research Agent

        Returns:
            AnalysisResult with identified pain points and opportunities
        """
        try:
            self.logger.info(f"Starting LangChain analysis for {company_info.name}")

            if self.use_mock:
                # Use mock analysis for demonstration
                return self._mock_analysis(company_info)
            else:
                # Use real LLM chains
                return self._llm_analysis(company_info)

        except Exception as e:
            self.logger.error(f"LangChain analysis failed for {company_info.name}: {str(e)}")
            raise

    def _mock_analysis(self, company_info: CompanyInfo) -> AnalysisResult:
        """Mock analysis for demonstration purposes"""

        # Generate mock pain points based on company characteristics
        pain_points = []
        opportunities = []

        if company_info.size.value == "startup":
            pain_points.append(PainPoint(
                category="growth",
                description="Scaling challenges and resource constraints",
                severity=PainPointSeverity.HIGH,
                evidence=["Startup stage company", "Limited resources"],
                potential_solution="Scalable solutions and growth support"
            ))

            opportunities.append(Opportunity(
                category="expansion",
                description="Market expansion opportunities",
                priority=OpportunityPriority.HIGH,
                evidence=["Startup growth phase", "Market potential"],
                potential_value="High growth potential"
            ))

        elif company_info.size.value == "mid-market":
            pain_points.append(PainPoint(
                category="operations",
                description="Operational efficiency challenges",
                severity=PainPointSeverity.MEDIUM,
                evidence=["Mid-market company", "Growth phase"],
                potential_solution="Process optimization solutions"
            ))

            opportunities.append(Opportunity(
                category="efficiency",
                description="Operational optimization opportunities",
                priority=OpportunityPriority.MEDIUM,
                evidence=["Mid-market size", "Growth potential"],
                potential_value="Moderate efficiency gains"
            ))

        # Industry-specific analysis
        if company_info.industry.lower() == "manufacturing":
            pain_points.append(PainPoint(
                category="sustainability",
                description="Sustainability compliance challenges",
                severity=PainPointSeverity.MEDIUM,
                evidence=["Manufacturing industry", "Regulatory requirements"],
                potential_solution="Sustainability and compliance solutions"
            ))

            opportunities.append(Opportunity(
                category="automation",
                description="Automation and digitization opportunities",
                priority=OpportunityPriority.HIGH,
                evidence=["Manufacturing industry", "Technology adoption"],
                potential_value="Significant efficiency improvements"
            ))

        elif company_info.industry.lower() == "saas":
            pain_points.append(PainPoint(
                category="customer_acquisition",
                description="Customer acquisition cost challenges",
                severity=PainPointSeverity.HIGH,
                evidence=["SaaS industry", "Competitive market"],
                potential_solution="Customer acquisition optimization"
            ))

            opportunities.append(Opportunity(
                category="ai_integration",
                description="AI/ML integration opportunities",
                priority=OpportunityPriority.HIGH,
                evidence=["SaaS industry", "Technology focus"],
                potential_value="Competitive advantage"
            ))

        # Generate industry insights
        industry_insights = [
            f"{company_info.industry} companies are increasingly focused on digital transformation",
            "Market consolidation is creating partnership opportunities",
            "Customer expectations are driving innovation in the industry"
        ]

        # Determine recommended approach
        high_severity_pains = [p for p in pain_points if p.severity in [PainPointSeverity.HIGH, PainPointSeverity.CRITICAL]]
        high_priority_opps = [o for o in opportunities if o.priority in [OpportunityPriority.HIGH, OpportunityPriority.URGENT]]

        if high_severity_pains:
            recommended_approach = "pain_point_focused"
        elif high_priority_opps:
            recommended_approach = "opportunity_focused"
        else:
            recommended_approach = "relationship_building"

        # Calculate confidence score
        confidence_score = 0.7  # Mock confidence score

        return AnalysisResult(
            company_name=company_info.name,
            pain_points=pain_points,
            opportunities=opportunities,
            industry_insights=industry_insights,
            recommended_approach=recommended_approach,
            confidence_score=confidence_score
        )

    def _llm_analysis(self, company_info: CompanyInfo) -> AnalysisResult:
        """Real LLM analysis using LangChain chains"""

        # Convert company info to string for LLM
        company_info_str = company_info.model_dump_json()

        # Run pain point analysis
        pain_point_result = self.pain_point_chain.run(company_info=company_info_str)

        # Run opportunity analysis
        opportunity_result = self.opportunity_chain.run(company_info=company_info_str)

        # Run strategy analysis
        strategy_result = self.strategy_chain.run(
            pain_points=pain_point_result,
            opportunities=opportunity_result
        )

        # Parse results and create AnalysisResult
        # Note: In a real implementation, you'd parse the LLM output
        # and convert it to structured data

        return AnalysisResult(
            company_name=company_info.name,
            pain_points=[],  # Would parse from pain_point_result
            opportunities=[],  # Would parse from opportunity_result
            industry_insights=["LLM-generated insights"],
            recommended_approach="pain_point_focused",  # Would parse from strategy_result
            confidence_score=0.8
        )

    def get_status(self) -> Dict[str, Any]:
        """Return agent status for monitoring"""
        return {
            "agent_id": self.agent_id,
            "status": "ready",
            "framework": "langchain",
            "use_mock": self.use_mock,
            "llm_available": self.llm is not None,
            "chains_configured": len(self._get_chain_names()) if not self.use_mock else 0
        }

    def _get_chain_names(self) -> List[str]:
        """Get list of configured chain names"""
        if self.use_mock:
            return []
        return ["pain_point_chain", "opportunity_chain", "strategy_chain"]

# Example usage and testing
if __name__ == "__main__":
    print("=== LangChain Analysis Agent Demo ===\n")

    # Create agent
    analysis_agent = LangChainAnalysisAgent(use_mock=True)

    # Test with mock company data
    from langchain_models import CompanyInfo, CompanySize

    test_company = CompanyInfo(
        name="Acme Corporation",
        industry="Manufacturing",
        size=CompanySize.MID_MARKET,
        location="Chicago, IL",
        website="https://acmecorp.com",
        description="Leading manufacturer of industrial equipment",
        recent_news=["Expansion into European markets", "New sustainability initiative"],
        key_contacts=[{"name": "Sarah Johnson", "title": "CEO", "email": "sarah@acmecorp.com"}]
    )

    try:
        analysis_result = analysis_agent.analyze_company(test_company)

        print(f"Analysis for: {analysis_result.company_name}")
        print(f"Confidence Score: {analysis_result.confidence_score:.2f}")
        print(f"Recommended Approach: {analysis_result.recommended_approach}")

        print(f"\nPain Points ({len(analysis_result.pain_points)}):")
        for i, pain_point in enumerate(analysis_result.pain_points, 1):
            print(f"  {i}. {pain_point.description}")
            print(f"     Category: {pain_point.category}")
            print(f"     Severity: {pain_point.severity}")
            print(f"     Evidence: {', '.join(pain_point.evidence)}")

        print(f"\nOpportunities ({len(analysis_result.opportunities)}):")
        for i, opportunity in enumerate(analysis_result.opportunities, 1):
            print(f"  {i}. {opportunity.description}")
            print(f"     Category: {opportunity.category}")
            print(f"     Priority: {opportunity.priority}")
            print(f"     Evidence: {', '.join(opportunity.evidence)}")

        print(f"\nIndustry Insights:")
        for insight in analysis_result.industry_insights:
            print(f"  • {insight}")

    except Exception as e:
        print(f"❌ Analysis failed: {str(e)}")

    print("\n" + "="*50 + "\n")

    # Show agent status
    status = analysis_agent.get_status()
    print("Agent Status:")
    for key, value in status.items():
        print(f"  {key}: {value}")


Let’s walk through this **LangChain Analysis Agent** step by step so you fully understand what it’s doing and what you should learn from it.

---

# 🟦 What This Agent Does

The **LangChainAnalysisAgent** takes structured company info (from your ResearchAgent) and produces an **AnalysisResult** with:

* Pain points
* Opportunities
* Industry insights
* A recommended sales approach (pain-point, opportunity, or relationship)
* Confidence score

It’s the same role as your earlier `AnalysisAgent`, but rebuilt on **LangChain’s primitives**.

---

# 🟦 Key Components

### 1. **LLM Setup**

```python
if use_mock:
    self.llm = None
else:
    self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=1000)
```

* `use_mock=True` means it won’t call the real OpenAI API — just use hand-coded mock logic.
* If `use_mock=False`, it spins up a LangChain `ChatOpenAI` instance.

👉 **Lesson:** Good practice to have a mock mode → lets you develop/test without burning tokens or needing API keys.

---

### 2. **Prompt Templates**

```python
self.pain_point_template = PromptTemplate(...)
self.opportunity_template = PromptTemplate(...)
self.strategy_template = PromptTemplate(...)
```

Each template is structured and instructs the LLM to produce outputs in a predictable format.

👉 **Lesson:** Instead of writing raw prompts inline, encapsulate them as **PromptTemplates**. This makes them reusable and easier to iterate on.

---

### 3. **Chains**

```python
self.pain_point_chain = LLMChain(llm=self.llm, prompt=self.pain_point_template)
```

* A `Chain` in LangChain = LLM + Prompt + (optionally) OutputParser.
* Here you have three chains: **pain point**, **opportunity**, **strategy**.

👉 **Lesson:** Break complex workflows into **composable chains** instead of one mega-prompt.

---

### 4. **Mock Analysis Logic**

```python
if company_info.size.value == "startup":
    pain_points.append(PainPoint(...))
    opportunities.append(Opportunity(...))
```

This hard-codes analysis logic for different company sizes/industries.

* Startups get “scaling challenges” + “expansion opportunities.”
* Manufacturing companies get “sustainability compliance” + “automation opportunities.”

👉 **Lesson:** Mocks let you simulate LLM behavior so your pipeline can run end-to-end *before* you wire in real AI.

---

### 5. **Real LLM Analysis**

```python
pain_point_result = self.pain_point_chain.run(company_info=company_info_str)
opportunity_result = self.opportunity_chain.run(company_info=company_info_str)
strategy_result = self.strategy_chain.run(...)
```

* Each chain runs independently.
* The strategy chain combines pain points + opportunities.

👉 **Lesson:** This is how you **compose multiple chains** for multi-step reasoning.

---

### 6. **Structured Outputs**

The agent returns an `AnalysisResult` object:

```python
return AnalysisResult(
    company_name=company_info.name,
    pain_points=pain_points,
    opportunities=opportunities,
    industry_insights=industry_insights,
    recommended_approach=recommended_approach,
    confidence_score=confidence_score
)
```

Instead of messy free text, everything is wrapped in a structured model (`Pydantic`).

👉 **Lesson:** Always enforce schemas on LLM output to make downstream automation reliable.

---

### 7. **Agent Status**

```python
def get_status(self):
    return { "agent_id": ..., "status": "ready", "framework": "langchain", ... }
```

This is for monitoring/observability.

👉 **Lesson:** Treat agents as microservices → each should have a clear “health check.”

---

# 🟦 Key Takeaways for You

1. **Prompt templates** make workflows reusable & maintainable.
2. **Chains** allow modular composition (pain points → opportunities → strategy).
3. **Mocks** let you test pipelines before integrating a real LLM.
4. **Structured outputs (Pydantic)** prevent chaos → enforce predictable JSON-like returns.
5. **Agent status** = observability hook → good for orchestrators & dashboards.

---

⚡ The big picture: This code shows how LangChain replaces your manual `if/else` orchestrator logic with **chains and templates**, while still keeping reliability via **structured outputs**.




🚀 — let’s break down the **major LangChain features** you’re using in this `LangChainAnalysisAgent` and why they matter.

---

# 🟦 1. **PromptTemplate**

* **What it does:** Lets you define reusable, parameterized prompts with placeholders.
* **Why it matters:**

  * No need to hardcode giant strings everywhere.
  * Makes prompts maintainable and tweakable.
  * Forces you to think of prompts as modular building blocks.

👉 Without LangChain: You’d write raw strings and manually `.format()` them.
👉 With LangChain: Cleaner, reusable objects tied directly to LLM execution.

---

# 🟦 2. **LLMChain**

* **What it does:** Bundles together an LLM + a PromptTemplate (and optionally an output parser).
* **Why it matters:**

  * Turns prompts into **callable objects** you can run like functions.
  * Makes multi-step workflows modular.
  * Handles the formatting and LLM call boilerplate for you.

👉 Without LangChain: You’d manually build strings, call the API, and parse responses.
👉 With LangChain: `result = chain.run(company_info="Acme Corp")` → done.

---

# 🟦 3. **Structured Output (via Pydantic models)**

* **What it does:** Encapsulates results in predictable schemas (`AnalysisResult`, `PainPoint`, `Opportunity`).
* **Why it matters:**

  * Downstream agents don’t choke on free-form text.
  * Gives you machine-readable, validated JSON-like objects.
  * Prevents hallucination chaos by enforcing shape & fields.

👉 Without LangChain: You’d regex/split strings and hope the format is consistent.
👉 With LangChain: Typed dataclasses or Pydantic models → robust contracts.

---

# 🟦 4. **Mock Mode**

* **What it does:** Lets you simulate outputs without calling an LLM.
* **Why it matters:**

  * Test pipelines locally without burning tokens.
  * Debug logic separately from model behavior.
  * Build confidence before scaling.

👉 Without LangChain: You’d need custom `if/else` hacks.
👉 With LangChain: Integrated toggles + simple fallbacks.

---

# 🟦 5. **Agent-Like Structure (Chaining Multiple Steps)**

* **What it does:** Pain point analysis, opportunity analysis, and strategy generation are separate chains that run sequentially.
* **Why it matters:**

  * Decomposes one big fuzzy prompt into **small, controllable steps**.
  * Easier to debug: if strategy fails, you know pain point and opportunity worked fine.
  * Scales to more steps easily.

👉 Without LangChain: You’d have one huge, fragile prompt.
👉 With LangChain: Composable micro-prompts = cleaner reasoning.

---

# 🟦 6. **Observability (get\_status)**

* **What it does:** Returns agent ID, framework, and health status.
* **Why it matters:**

  * Helps orchestration and dashboards monitor agent readiness.
  * Moves you toward “agents as services” with health checks.

---

# 🟦 Big Picture Improvements

✅ **Less boilerplate:** No manual string building, API calling, or parsing.
✅ **Modularity:** PainPoint, Opportunity, and Strategy chains are plug-and-play.
✅ **Maintainability:** You can swap templates, models, or outputs without rewriting logic.
✅ **Reliability:** Structured results ensure downstream agents don’t break.
✅ **Scalability:** Easy to add more analysis chains later (e.g., SWOTAgent).

---

⚡ **Key takeaway:** LangChain isn’t magic — but it abstracts away the repetitive plumbing so you can focus on **workflow design, prompt strategy, and structured outputs**.






## ⚡ What I’d Add Before Moving On

1. **Parsing Real LLM Output**

   * Right now, `_llm_analysis()` returns empty lists for pain points and opportunities and just inserts placeholders .
   * In production, you’d want to parse the text returned by `pain_point_chain.run()` and map it back into your `PainPoint` and `Opportunity` objects.
   * This could be done with **LangChain’s structured output parsers** or even JSON-mode from newer OpenAI models.

2. **Confidence Score Calculation**

   * Currently hardcoded (0.7 in mock, 0.8 in LLM mode) .
   * You might want a heuristic like:

     * # of strong evidence items
     * # of high-severity pain points
     * Consistency across chains
   * Or, you could even ask the LLM to self-report a confidence measure.

3. **Extensibility Hooks**

   * The design is great for *core analysis*.
   * But you may later want to add specialized chains (e.g., *competitive threats*, *regulatory risks*, *financial health*).
   * With the current modular setup, you can drop in new chains easily.

4. **Error Handling for LLM Mode**

   * In `_llm_analysis`, if the model produces malformed text, the code might crash when parsing.
   * Adding try/except with fallbacks to a safe “relationship\_building” strategy would make the agent more resilient.

---

## 🟦 Bottom Line

The **Analysis Agent** is already a strong demo: modular, testable, and orchestration-ready.
The **next step** is making `_llm_analysis` fully production-ready by:

* parsing outputs → structured models,
* computing real confidence, and
* handling LLM quirks gracefully.
